# Create a Speech-Enabled Apps with Azure AI Services

### Build **AI Language** and  **AI Speech** Services on Azure Portal.

#### Step1. Create your own **AI Language** and  **AI Speech** Services to get the API KeyS and Endpoint.

In [18]:
QUESTION_ANSWERING_AI_KEY = '9251d385e873424abe9af031d509902a'
QUESTION_ANSWERING_AI_ENDPOINT = 'https://ces-language-service.cognitiveservices.azure.com/'

SPPECH_AI_KEY = 'a88d5c63922449d880e33ea4feb7f8c8'
SPEECH_AI_REGION = 'eastus2'

#### Step2. Install Azure AI Python SDK for **AI Speech** and **AI Language**.

In [19]:
!pip install azure-ai-language-questionanswering
!pip install azure-cognitiveservices-speech==1.30.0
!pip install playsound==1.2.2

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


#### Step3. Create an Speech Client driven by Audio File

In [20]:
import os
import azure.cognitiveservices.speech as speech_sdk

audio_path = os.getcwd() + '\\assets\\time.wav'
audio_config = speech_sdk.AudioConfig(filename=audio_path)

speech_client = speech_sdk.SpeechRecognizer(speech_sdk.SpeechConfig(SPPECH_AI_KEY, SPEECH_AI_REGION), audio_config)

#### Step4. Perform a Demonstration of its Functionality.

In [21]:
from playsound import playsound
response = speech_client.recognize_once_async().get().text
print('Response:', response)

# Play the original audio
playsound(audio_path)

Response: What time is it?


### Combine with QnA Service built with `./assets/basic-kb.tsv` Dataset.

#### Step1. Create an Question Answering Client

In [27]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.language.questionanswering import QuestionAnsweringClient

credential = AzureKeyCredential(QUESTION_ANSWERING_AI_KEY)
qna_client = QuestionAnsweringClient(endpoint=QUESTION_ANSWERING_AI_ENDPOINT, credential=credential)

#### Step2. Create an Question Answering Client driven by Microphone

In [28]:
audio_config = speech_sdk.AudioConfig(use_default_microphone=True)
speech_client = speech_sdk.SpeechRecognizer(speech_sdk.SpeechConfig(SPPECH_AI_KEY, SPEECH_AI_REGION), audio_config)

#### Step3. Start to Asking Question with Speech Client

In [29]:
input("Press 'Enter' to Start Recording...(ask some question about basic-kb.tsv)")
response = speech_client.recognize_once_async().get().text
print('Response:', response)

Press 'Enter' to Start Recording...(ask some question about basic-kb.tsv) 


Response: Why my URL is not extracting question answering pairs?


#### Step4. Generating Text Answer via QnA Clinet with Prebuilt Knowledge Base

In [33]:
response = qna_client.get_answers(question=response,
                                    project_name='QA',
                                    deployment_name='production')
for candidate in response.answers:
    print(candidate.answer)
    print("- Confidence: {}".format(candidate.confidence))
    print("- Source: {}".format(candidate.source))

See more details about [supported languages](https://docs.microsoft.com/azure/cognitive-services/qnamaker/overview/languages-supported).

If you have content from multiple languages, be sure to create a separate service for each language.
- Confidence: 0.0679
- Source: https://docs.microsoft.com/azure/cognitive-services/qnamaker/faqs


#### Step5. Synthesize Sound for Response Text via Speech Client

In [34]:
speech_config.speech_synthesis_voice_name = "en-GB-RyanNeural"
speech_synthesizer = speech_sdk.SpeechSynthesizer(speech_config)

speech_synthesizer.speak_text_async(candidate.answer).get()